<center><h1><b>3주차 ✅ 국가건강검진 데이터 EDA</b></h1></center>

---
----

<center><h2><b>데이터 가져오기</b></h2></center>


In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# df_raw_raw 행/열 전체 보기
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
# 데이터 불러오기
df_raw = pd.read_csv('data/국민건강보험공단_건강검진정보_20211229.CSV', encoding='cp949')[:50000]
df_raw.head()

In [ ]:
# 행/열 확인
df_raw.shape

In [ ]:
# 일부 정보 확인
df_raw.tail()

In [ ]:
# 일부 정보 확인
df_raw.sample(5)


In [ ]:
# 컬럼명 총 확인
df_raw.columns

In [ ]:
# 요약 정보
df_raw.info()


---
----

<center><h2><b>기술 통계</b></h2></center>


In [ ]:
# 요약 통계 (수치형)
df_raw.describe()


In [ ]:
# 요약 통계 (범주형)
df_raw.describe(include='object')

In [ ]:
# 결측치 확인
df_raw.isnull().sum()

In [ ]:
# 결측치 총계
df_raw.isnull().sum().sum()


In [ ]:
# 상관계수
# df_raw.corr().style.background_gradient(cmap='coolwarm')


In [ ]:
# 고유값의 개수
df_raw.nunique()


In [ ]:
# 범주형 타입으로 변경
# 성별, 연령대, 신장, 체중, 청력, 요단백, 흡연상태, 음주여부, 구강검진 수검여부, 치아우식증유무, 치석

obejects = [
    "시도코드", "성별코드", "연령대 코드(5세단위)", "신장(5Cm단위)", "체중(5Kg 단위)",
    "시력(좌)", "시력(우)", "청력(좌)", "청력(우)", "요단백", "흡연상태", "음주여부",
    "구강검진 수검여부", "치아우식증유무", "치석"]

for o in obejects:
    df_raw[o] = df_raw[o].astype('category')
    # df[o] = df[o].astype('str') # str 하면 object 타입으로 변경

In [ ]:
# 범주형 타입으로 잘 변경됐는지 확인
df_raw.dtypes


In [ ]:
# 개별 확인
type(df_raw['시도코드'].iloc[0])


In [ ]:
# 수치형 데이터 기술통계
df_raw.describe()


In [ ]:
# 범주형 데이터 기술통계
df_raw.describe(include='category')


In [ ]:
# 고유값 별개로 확인
for o in obejects:
    print(o)
    print(df_raw[o].unique())


In [ ]:
# 필요한 컬럼만
df = df_raw.drop(columns=['가입자 일련번호', "기준년도", '데이터 공개일자'])
df.head(1)

In [ ]:
df.describe().columns

In [ ]:
# 이상치 제거
for col in df.describe().columns:
    print(col)
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    df[col] = df[~(df[col] > q3+1.5*iqr)][col]

---
----

<center><h2><b>컬럼 이름 매핑</b></h2></center>


In [ ]:
# 시도코드 코드-도시이름 매핑하기
sido_raw = """
11:서울특별시
26:부산광역시
27:대구광역시
28:인천광역시
29:광주광역시
30:대전광역시
31:울산광역시
36:세종특별자치시
41:경기도
42:강원도
43:충청북도
44:충청남도
45:전라북도
46:전라남도
47:경상북도
48:경상남도
49:제주특별자치도
"""

sido_code = {}
for raw in sido_raw.split("\n"):
    if ':' in raw:
        key = int(raw.split(":")[0].strip())
        value = raw.split(":")[1].strip()
        sido_code[key] = value

sido_code

In [ ]:
# 시도코드 매핑하기
df['시도명'] = df['시도코드'].map(sido_code)
df.head(1)

In [ ]:
df['시도명'].value_counts()

In [ ]:
# 연령대 코드 빈도수 확인
df['연령대 코드(5세단위)'].value_counts().sort_index()

In [ ]:
# 연령대 코드 매핑
age_raw = """
1:0~4세
2:5~9세
3:10~14세
4:15~19세
5:20~24세
6:25~29세
7:30~34세
8:35~39세
9:40~44세
10:45~49세
11:50~54세
12:55~59세
13:60~64세
14:65~69세
15:70~74세
16:75~79세
17:80~84세
18:85세+
"""

age_code = {}
for raw in age_raw.split("\n"):
    if ':' in raw:
        key = int(raw.split(":")[0].strip())
        value = raw.split(":")[1].strip()
        age_code[key] = value

age_code

In [ ]:
# 연령대 매핑
df['연령대'] = df['연령대 코드(5세단위)'].map(age_code)
df.head(2)

In [ ]:
# 성별코드 매핑
df.loc[df['성별코드'] == 2, '성별'] = '여성'
df.loc[df['성별코드'] == 1, '성별'] = '남성'

In [ ]:
df.head(2)

---
----

<center><h2><b>테이블 교차</b></h2></center>


In [ ]:
df.head()

In [ ]:
pd.crosstab(df['시력(좌)'], [df['시력(우)']])


In [ ]:
pd.crosstab(df['청력(좌)'], [df['청력(우)']])


In [ ]:
df.columns

In [ ]:
px.histogram(df.groupby('총 콜레스테롤').mean()[['LDL 콜레스테롤', 'HDL 콜레스테롤']])


---
----

<center><h2><b>시각화 그래프</b></h2></center>


In [ ]:
df.columns

In [ ]:
ssa_hgt = pd.crosstab([df['시도명'], df['연령대']], [
                      df['신장(5Cm단위)'], df['성별']]).sort_index()
ssa_hgt.head()


In [ ]:
df.corr().style.background_gradient(cmap='coolwarm')

-----
### 신장
- 성별을 기준으로 한 신장
- 연령대를 기준으로 한 신장
- 신장과 체중
- 신장과 허리둘레

In [ ]:
# 필요한 컬럼
cols = ['시도명', '성별', '연령대', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레',
        '총 콜레스테롤', '트리글리세라이드', 'HDL 콜레스테롤', 'LDL 콜레스테롤', '혈색소', '요단백',
        '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마 지티피', '흡연상태', '음주여부',
        '구강검진 수검여부', '치아우식증유무', '치석', ]


In [ ]:
# 신장을 기준으로 정렬
hgt = df.sort_values('신장(5Cm단위)')
hgt = hgt[cols]
hgt.head(2)


In [ ]:
# 신장을 기준으로 기술통계
hgt.describe()

In [ ]:
# 체중 - 신장 (boxplot)
px.box(hgt, x='신장(5Cm단위)', y='체중(5Kg 단위)', points='all',
       facet_col="성별", color="성별", boxmode="overlay",
       title='신장에 따른 남성/여성의 체중 통계' )


In [ ]:
# 신장 - 체중 (성별)
# hisfunc = "count" | "sum" | "avg" | "min" | "max"
# barmode = "stack" | "group" | "overlay" | "relative"
fig = px.histogram(hgt, x='신장(5Cm단위)', y='체중(5Kg 단위)', color='성별',
        barmode='group', title='신장에 따른 남성/여성의 평균 체중', histfunc='avg', text_auto=True,)
fig.update_xaxes(type='category', ticksuffix='cm')
fig.update_yaxes(ticksuffix='kg')
fig.show()

In [ ]:
# 신장 - 체중 (연령대)
fig = px.histogram(
    hgt, x='신장(5Cm단위)', y='체중(5Kg 단위)', color='연령대',
    barmode='relative', title='신장에 따른 연령대별 평균 체중', histfunc='avg', text_auto=True,
    facet_col='연령대', facet_col_wrap=3,
    width=1200, height=800,
    # category_orders={'체중': []}
)
fig.update_xaxes(type='category', ticksuffix='cm',)
fig.update_yaxes(ticksuffix='kg', )
fig.show()


In [ ]:
# 체중 - 허리둘레 (boxplot)
px.box(hgt, x='신장(5Cm단위)', y='허리둘레', points='all',
       facet_col="성별", color="성별", boxmode="overlay",
       title='신장에 따른 남성/여성의 허리둘레 통계' )


In [ ]:
# 신장 - 허리둘레 (성별)
fig = px.histogram(hgt, x='신장(5Cm단위)', y='허리둘레', color='성별',
        barmode='group', title='신장에 따른 남성/여성의 평균 허리둘레', histfunc='avg', text_auto=True,)
fig.update_xaxes(type='category', ticksuffix='cm')
fig.update_yaxes(ticksuffix='cm')
fig.show()

In [ ]:
# 신장 - 허리둘레 (연령대)
fig = px.histogram(
    hgt, x='신장(5Cm단위)', y='허리둘레', color='연령대',
    barmode='relative', title='신장에 따른 연령대별 평균 허리둘레', histfunc='avg', text_auto=True,
    facet_col='연령대', facet_col_wrap=3,
    width=1200, height=800,
)
fig.update
fig.update_xaxes(type='category', ticksuffix='cm',)
fig.update_yaxes(ticksuffix='cm', )
fig.show()


-----
### 체중
- 성별을 기준으로 한 체중
- 연령대를 기준으로 한 체중
- 콜레스테롤, 혈압 등등
- 음주여부, 흡연여부 등등

In [ ]:
# 체중을 기준으로 정렬
wgt = df.sort_values('체중(5Kg 단위)')
wgt = wgt[cols]
wgt.head(2)


In [ ]:
# 체중 - 허리둘레 (boxplot)
# boxmode = 'overlay' | 'group'
fig = px.box(wgt, x='체중(5Kg 단위)', y='허리둘레', points='all',
       facet_col="성별", color="성별", boxmode="overlay",
       title='체중에 따른 남성/여성의 허리둘레 통계')
fig.update_yaxes(ticksuffix='cm')
fig.show()

In [ ]:
# 체중 - 허리둘레 (성별)
fig = px.histogram(wgt, x='체중(5Kg 단위)', y='허리둘레', color='성별',
        barmode='group', title='체중에 따른 남성/여성의 평균 허리둘레', histfunc='avg', text_auto=True,)
fig.update_xaxes(type='category', ticksuffix='kg')
fig.update_yaxes(ticksuffix='cm')
fig.show()


In [ ]:
# 체중 - 허리둘레 (연령대)
fig = px.histogram(
    wgt, x='체중(5Kg 단위)', y='허리둘레', color='연령대',
    barmode='relative', title='체중에 따른 연령대별 평균 허리둘레', histfunc='avg', text_auto=True,
    facet_col='연령대', facet_col_wrap=3,
    width=1200, height=800,
)
fig.update
fig.update_xaxes(type='category', ticksuffix='kg',)
fig.update_yaxes(ticksuffix='cm', )
fig.show()


In [ ]:
# 체중 - 총 콜레스테롤 (boxplot)
fig = px.box(wgt, x='체중(5Kg 단위)', y='총 콜레스테롤', points='all',
       facet_col="성별", color="성별", boxmode="overlay",
       title='체중에 따른 남성/여성의 총 콜레스테롤 통계')
fig.update_yaxes(ticksuffix='mg/dl')
fig.show()

In [ ]:
# 콜레스테롤 히스토그램
px.histogram(wgt, x='총 콜레스테롤', nbins=200)

In [ ]:
# 총 콜레스테롤 왜도/첨도
wgt['총 콜레스테롤'].skew(), wgt['총 콜레스테롤'].kurt()

In [ ]:
# 체중 - 총 콜레스테롤 (성별)
fig = px.histogram(wgt, x='체중(5Kg 단위)', y='총 콜레스테롤', color='성별',
        barmode='group', title='체중에 따른 남성/여성의 평균 총 콜레스테롤 지수', histfunc='avg', text_auto=True,)
fig.update_xaxes(type='category', ticksuffix='kg')
fig.update_yaxes(ticksuffix='mg/dl')
fig.show()

In [ ]:
# 체중 - 총 콜레스테롤 (연령대)
fig = px.histogram(
    wgt, x='체중(5Kg 단위)', y='총 콜레스테롤', color='연령대',
    barmode='relative', title='체중에 따른 연령대별 평균 총 콜레스테롤 지수', histfunc='avg', text_auto=True,
    facet_col='연령대', facet_col_wrap=3,
    width=1200, height=800,
)
fig.update
fig.update_xaxes(type='category', ticksuffix='kg',)
fig.update_yaxes(ticksuffix='mg/dl', )
fig.show()


In [ ]:
# 채중 - 요단백 (결측치 제거)
wgt_yo = wgt.dropna(how='any', axis=0)
wgt_yo.head(2)

In [ ]:
# 체중 - 콜레스테롤 (boxplot/요단백)
fig = px.box(wgt_yo, x='체중(5Kg 단위)', y='총 콜레스테롤', points='all',
       color="요단백", boxmode="overlay",
       title='체중에 따른 요단백 등급별 총 콜레스테롤 통계')
fig.update_yaxes(ticksuffix='cm')
fig.show()

In [ ]:
# 체중 - 콜레스테롤 (요단백)
fig = px.histogram(wgt_yo, x='체중(5Kg 단위)', y='총 콜레스테롤', color='요단백',
        barmode='group', title='체중에 따른 요단백 등급별 평균 콜레스테롤 지수', histfunc='avg', text_auto=True,)
fig.update_xaxes(type='category', ticksuffix='kg')
fig.update_yaxes(ticksuffix='mg/dl')
fig.show()

In [ ]:
# 요단백 등급별 count
px.histogram(wgt_yo, x = '요단백', barmode='relative', color='요단백', text_auto=True)

In [ ]:
# 체중 - 흡연상태 (성별)
fig = px.histogram(wgt, x='체중(5Kg 단위)', y='흡연상태', color='성별',
        barmode='group', title='체중에 따른 남성/여성 흡연자 수', histfunc='count', text_auto=True,
        category_orders={'흡연상태': [1, 2, 3]})
fig.update_xaxes(type='category', ticksuffix='kg')
fig.update_yaxes(ticksuffix='명')
fig.show()

In [ ]:
# 체중 - 허리둘레 (연령대)
fig = px.histogram(
    wgt, x='체중(5Kg 단위)', y='흡연상태', color='연령대',
    barmode='relative', title='체중에 따른 연령대별 흡연자 수', histfunc='count', text_auto=True,
    facet_col='연령대', facet_col_wrap=3,
    width=1200, height=800,
)
fig.update
fig.update_xaxes(type='category', ticksuffix='kg',)
fig.update_yaxes(ticksuffix='명', )
fig.show()


In [ ]:
wgt.head()

In [ ]:
# 체중 - 흡연상태 (성별)
fig = px.histogram(wgt_, x='체중(5Kg 단위)', y='흡연상태', color='흡연상태',
        barmode='group',
         title='체중에 따른 흡연상태 (1:안핀다 2:끊었다 3:피운다)',
        histfunc='count',
         text_auto=True,
        category_orders={'흡연상태': [1, 2, 3]}, )
fig.update_xaxes(type='category', ticksuffix='kg')
fig.update_yaxes(ticksuffix='명')
fig.show()

In [ ]:
# 연령대별 - 음주여부
px.histogram(wgt_.sort_values('연령대'), x='연령대', barmode='relative', color='음주여부',
             text_auto=True, title='연령대별 음주여부')


In [ ]:
# 체중 - 음주여부
fig = px.histogram(wgt_, x='체중(5Kg 단위)', color='음주여부',
        barmode='group', title='체중에 따른 음주여부 (1:마신다 0:안마신다)', histfunc='count', text_auto=True,)
fig.update_xaxes(type='category', ticksuffix='kg')
fig.update_yaxes(ticksuffix='명')
fig.show()

---
----

<center><h2><b>지도 시각화</b></h2></center>


In [ ]:
import json

state_geo = './data/TL_SCCO_CTPRVN.json'
state_geo
 
 # 시도기준 경계 JSON 파일을 들고 온다.
with open(state_geo, encoding='utf-8') as file:
    sido_map = json.load(file)


In [ ]:
# 올바른 값이 들어있는지 확인
sido_map['features'][0]['properties']['CTP_KOR_NM']

In [ ]:
# 시도명으로 평균치를 그룹핑 한다.
sido = df.groupby('시도명', as_index=False).mean()
sido.head(2)


In [ ]:
# 시도기준 위도/경도
# locs = { 
#     '경기도':   (37.567167, 127.190292),
#     '서울특별시':     (37.487935, 126.857758),
#     '부산광역시':    (35.198362, 129.053922),
#     '경상북도':  (36.248647, 128.664734),
#     '경상남도': (35.259787, 128.664734),
#     '인천광역시':  (37.469221, 126.573234),
#     '대구광역시': (35.798838, 128.583052),
#     '충청남도': (36.557229, 126.779757),
#     '전라남도':  (34.819400, 126.893113),
#     '전라북도':  (35.716705, 127.144185),
#     '대전광역시':   (36.321655, 127.378953),
#     '강원도': (37.555837, 128.209315),
#     '광주광역시': (35.126033, 126.831302),
#     '울산광역시': (35.519301, 129.239078),
#     '충청북도': (36.628503, 127.929344),
#     '세종특별자치시':    (36.48, 127.29),
#     '제주특별자치도':   (33.364805, 126.542671),
# }

In [ ]:
# 위도경도 매핑

locs = {
    '경기도': (37.95, 126.95),
    '서울특별시': (37.58, 126.7),
    '부산광역시':(35.198362, 129.053922),
    '경상북도':  (36.63, 128.46),
    '경상남도': (35.5, 128),
    '인천광역시':  (37.5, 125.8),
    '대구광역시': (35.96, 128.32),
    '충청남도': (36.69, 126),
    '전라남도':  (34.819400, 126.893113),
    '전라북도':  (35.86, 126.85),
    '대전광역시':   (36.321655, 127.378953),
    '강원도': (37.88, 128),
    '광주광역시': (35.28, 126.49),
    '울산광역시': (35.8, 129.5),
    '충청북도': (37.19, 127.50),
    '세종특별자치시':    (36.7, 127.07),
    '제주특별자치도':   (33.62, 126.11),
}

locs


In [ ]:
# 숫자만 뽑아내기
# import re
# re.findall('\d+', str(sido.loc[dfdf['시도명'] == '부산광역시', '허리둘레'])) 

(sido.loc[sido['시도명'] == '부산광역시', '허리둘레']).to_string().split(' ')[-1]


In [ ]:
sido.head()

In [ ]:
sido_s = df.groupby(by=['시도명', '성별']).mean()
sido_s.head()

In [ ]:
sido_s.loc[('서울특별시', '여성'), '허리둘레']


In [ ]:
sido_h = sido.groupby('시도명').mean()
sido_h.head()


In [ ]:
# 지도 그려보기
import folium
from folium.features import DivIcon

# 도화지 그리기
m = folium.Map(location=[35.8, 128.071503], zoom_start=7,)

# Choropleth 맵을 그린다.
ch = folium.Choropleth(
    geo_data=sido_map,
    name='choropleth',
    data=sido,
    columns=['시도명', '허리둘레'],  # 매핑되는 값
    key_on='feature.properties.CTP_KOR_NM',  # 시도명 - CTP_KOR_NM 이 맞아야 지도 매핑 가능
    fill_color='YlGnBu',  # 지도 컬러 설정
    fill_opacity=0.7,  # 지도 투명
    line_opacity=1,  # 선 투명
    line_weight=1.5,  # 선 굵기
    line_color='#000',  # 선 색
    legend_name='허리둘레',  # 범례 이름
    #  highlight=True, # 하이라이트 설정
).add_to(m)

# 맵에 text를 추가한다.
for key, value in locs.items():
    # Marker 를 이용해서 텍스트 추가
    folium.map.Marker(
        [value[0], value[1]],  # 위/경도 위치에 text가 추가된다.
        icon=DivIcon(
            icon_size=(250, 36),
            icon_anchor=(0, 0),

            # div 태그 추가
            html='<div style="\
                font-size: 1.1rem;\
                color: black;\
                background-color:rgba(255, 255, 255, 0.3);\
                width:85px;\
                text-align:center;\
                margin:0px;">'
            + "<b>"
            + key + ': ' + str(round(sido_h.loc[key, '허리둘레'], 2))
            # + "<br/><span style='color: red; margin: 0px;'>"
            # + (sido_s.loc[sido['시도명'] == key, '허리둘레']).to_string().split(' ')[-1] + '</span>'
            + "<br/><span style='color: red; margin: 0px;'>여성: "
            + str(round(sido_s.loc[(key, '여성'), '허리둘레'], 2)) + '</span>'
            + "<br/><span style='color: blue; margin: 0px;'>남성: "
            + str(round(sido_s.loc[(key, '남성'), '허리둘레'], 2)) + '</span>'
            + '</b></div>',
        )).add_to(m)

# layer에 지도 추가
folium.LayerControl().add_to(m)

# 타이틀 추가
folium.map.Marker(
    [34.2, 126.8],
    icon=DivIcon(
        icon_size=(500,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 3rem; background-color:black;color:white;text-align:center;">'
        +'<b>도시에 따른 성별 허리둘레 지수<b></div>',
        )
    ).add_to(m)

# 툴팁 추가
# ch.geojson.add_child(
#     folium.features.GeoJsonTooltip(['CTP_KOR_NM'], labels=False)
# )

# html 파일로 만들기
# m.save('save.html')

m  # 지도를 그린다.


In [ ]:
# folium 맵 사진으로 저장하기
# conda install -c conda-forge pyppeteer
from folium import utilities
from pyppeteer import launch

html = m.get_root().render()
browser = await launch(headless=True)

page = await browser.newPage()
with utilities.temp_html_filepath(html) as fname:
    await page.goto('file://{path}'.format(path=fname))

img_data = await page.screenshot({'path': f'output/out.png', 'fullPage': 'true', })
await browser.close()

-----
### 지도 만드는 함수

In [ ]:
from folium import utilities
from pyppeteer import launch

# 만든 지도를 png 파일로 캡쳐해서 저장하는 함수
# @asyncio.coroutine
async def map_to_png(target, m):
    html = m.get_root().render()
    browser = await launch(headless=True)

    page = await browser.newPage()
    with utilities.temp_html_filepath(html) as fname:
        await page.goto('file://{path}'.format(path=fname))

    img_data = await page.screenshot({'path': f'output/out_{target}.png', 'fullPage': 'true', })
    await browser.close()

In [ ]:
from folium.features import DivIcon
import folium



# 지도 그리는 함수
def load_map(target):
    m = folium.Map(location=[35.8, 128.071503], zoom_start=7,)

    # Choropleth 맵을 그린다.
    ch = folium.Choropleth(
        geo_data=sido_map,
        name='choropleth',
        data=sido,
        columns=['시도명', target],  # 매핑되는 값
        key_on='feature.properties.CTP_KOR_NM',  # 시도명 - CTP_KOR_NM 이 맞아야 지도 매핑 가능
        fill_color='YlGnBu',  # 지도 컬러 설정
        fill_opacity=0.7,  # 지도 투명
        line_opacity=1,  # 선 투명
        line_weight=1.5,  # 선 굵기
        line_color='#000',  # 선 색
        legend_name=target,  # 범례 이름
    ).add_to(m)

    # 맵에 text를 추가한다.
    for key, value in locs.items():
        # Marker 를 이용해서 텍스트 추가
        folium.map.Marker(
            [value[0], value[1]],  # 위/경도 위치에 text가 추가된다.
            icon=DivIcon(
                icon_size=(250, 36),
                icon_anchor=(0, 0),

                # div 태그 추가
                html='<div style="\
                    font-size: 1.1rem;\
                    color: black;\
                    background-color:rgba(255, 255, 255, 0.3);\
                    width:85px;\
                    text-align:center;\
                    margin:0px;">'
                + "<b>"
                + key + ': ' + str(round(sido_h.loc[key, target], 2))
                + "<br/><span style='color: red; margin: 0px;'>여성: "
                + str(round(sido_s.loc[(key, '여성'), target], 2)) + '</span>'
                + "<br/><span style='color: blue; margin: 0px;'>남성: "
                + str(round(sido_s.loc[(key, '남성'), target], 2)) + '</span>'
                + '</b></div>',
            )).add_to(m)

    # layer에 지도 추가
    folium.LayerControl().add_to(m)

    # title 추가
    folium.map.Marker(
    [34.2, 126.8],
    icon=DivIcon(
        icon_size=(500,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 3rem; background-color:black;color:white;text-align:center;">'
        +f'<b>도시에 따른 성별 {target} 지수<b></div>',
        )
    ).add_to(m)

    return m

    

In [ ]:
# 함수 test 허리둘레 지도 시각화
target = '허리둘레'
waste_m = load_map(target)
await map_to_png(target, waste_m)


In [ ]:
# 수치형 변수 모두 지도 시각화
for col in sido.columns[1:]: 
    m = load_map(col)
    await map_to_png(col, m)
